In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
from datetime import (datetime, timedelta)

from marthas_dashboard.api import API
from marthas_dashboard.views import tools

In [2]:
TIME_FMT = "%Y-%m-%d %H:%M:%S"
api = API()

## Room Compare

Tabular view of rooms in building at timestamp

In [3]:
# api.buildings()

In [4]:
def merge_tables(points, rooms, vals):
    """Takes three dfs generated by api queries and merges together."""
    df = pd.merge(points, rooms, left_on='roomid', right_on='id', suffixes=('_point', '_room'))
    df = pd.merge(df, vals, left_on='id_point', right_on='pointid', )
    return df

def hacky_tagging(df):
    """Attempt to tag points in DF: To be replaced with Zephyr's tags"""

    df['tag'] = 'none'  # add tag column (set to 'none' by default)

    df_valve = df[df['description'].str.find('VALVE') > 0]  # tag 'valve'
    df.loc[df_valve.index, 'tag'] = 'valve'

    df_temp = df[df['name_point'].str.find('.RM') > 0]  # tag 'temp1'
    df.loc[df_temp.index, 'tag'] = 'temp1 (RM)'

    df_temp = df[df['name_point'].str.find('.RMT') > 0]  # tag 'temp2'
    df.loc[df_temp.index, 'tag'] = 'temp2 (RMT)'
    return df

In [5]:
def pivot_table_around_tags(df, idx):
    """Pivot table so tags become columns"""

    # Get values where tag is not 'none'
    df = df.query('tag != "none"')

    # call pivot_on_tag with each
    tags = ['valve', 'temp1 (RM)', 'temp2 (RMT)']
    frames = [pivot_on_tag(df, t) for t in tags]
    final_df = pd.concat(frames, axis=1).reset_index()
    return final_df


def pivot_on_tag(df, tag):
    """Helper function to 'pivot_table_around_tags'

    :param df: tagged df (with all rooms, points, and vals for given building/timestamp)
    :param tag: tag to extact (eg, "valve")
    :return: filtered/pivoted df: <index = name_room, columns = [pointid_{tag}, {tag}]>
    """
    try:
        df = (df.query('tag == "{}"'.format(tag))
              .pivot_table(index=['name_room', 'pointid'], columns='tag', values='pointvalue')
              .reset_index().rename_axis(None, axis=1)
              .rename({'pointid': 'pointid_{}'.format(tag)}, axis='columns'))
    except DataError:
        df = pd.DataFrame(columns=['name_room', tag])
    return df.set_index('name_room')

In [6]:
#43

In [7]:
searches = {
    'building': '47',
    'date': '2017-08-18',
    'timestamp': '00:00:00'}

In [8]:
from marthas_dashboard.tools import get_room_comparison_results
get_room_comparison_results(searches)

""


In [70]:
keywords = searches

building_id, date, timestamp = keywords["building"], keywords["date"], keywords["timestamp"]
full_timestamp = date + " " + timestamp

# full_timestamp = datetime.strptime((date + " " + timestamp), TIME_FMT)

# Make api calls (maybe this should be a single SQL call)
rooms = api.building_rooms(building_id)
points = api.building_points(building_id)
vals = api.building_values_at_time(building_id, full_timestamp)

query_frames = [rooms, points, vals]
if any(len(x)==0 for x in query_frames):
    df = pd.DataFrame()

# Remove dummy rooms
# rooms = rooms[rooms['name'].str.find('_Dummy_') < 0]

# Merge dfs together, attempt to tag points, pivot so tags become columns
df = merge_tables(points, rooms, vals)

if len(df)==0:
    df = pd.DataFrame()
    
df = hacky_tagging(df)
df = pivot_table_around_tags(df, 'name_room')

TypeError: Must pass list-like as `names`.

In [68]:
df = merge_tables(points, rooms, vals)

In [69]:
df

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
0,Electricity (kWh),NaN,1667,Goodhue - Electricity,1,4115,61,47,61,Goodhue_Dummy_Room,...,4115,Goodhue - Electricity,1667,Goodhue - Electricity,2017-08-18,22,float,kWh,2017-08-18,00:00


In [20]:
query_frames = [rooms, points, vals]

In [21]:
any(len(x)==0 for x in query_frames)

True

In [161]:
if any(len(x)==0 for x in query_frames):
    print("here")

here


In [114]:
query_frames = [rooms, points, vals]

In [127]:
[len(x)==0 for x in query_frames]

[True, False, True]

In [126]:
any(len(x)==0 for x in query_frames)

True

In [102]:
df

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time,tag


In [96]:
tag = 'valve'
(df.query('tag == "{}"'.format(tag))
 .pivot_table(index=['name_room', 'pointid'], columns='tag', values='pointvalue'))

name_room,pointid


In [55]:
tag = 'valve'
df.query('tag == "{}"'.format(tag))

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time,tag


In [28]:
tags = ['valve', 'temp1 (RM)', 'temp2 (RMT)']
frames = [pivot_on_tag(df, t) for t in tags]

TypeError: Must pass list-like as `names`.

In [65]:
from pandas.core.base import DataError

def pivot_table_around_tags(df, idx):
    """Pivot table so tags become columns"""

    # Get values where tag is not 'none'
    df = df.query('tag != "none"')

    # call pivot_on_tag with each
    tags = ['valve', 'temp1 (RM)', 'temp2 (RMT)']
    frames = [pivot_on_tag(df, t) for t in tags]
    final_df = pd.concat(frames, axis=1).reset_index()
    return final_df


def pivot_on_tag(df, tag):
    """Helper function to 'pivot_table_around_tags'

    :param df: tagged df (with all rooms, points, and vals for given building/timestamp)
    :param tag: tag to extact (eg, "valve")
    :return: filtered/pivoted df: <index = name_room, columns = [pointid_{tag}, {tag}]>
    """
    try:
        df = (df.query('tag == "{}"'.format(tag))
              .pivot_table(index=['name_room', 'pointid'], columns='tag', values='pointvalue')
              .reset_index().rename_axis(None, axis=1)
              .rename({'pointid': 'pointid_{}'.format(tag)}, axis='columns'))
    except DataError:
        df = pd.DataFrame(columns=['name_room', tag])
    return df.set_index('name_room')


In [24]:
df

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time,tag


In [52]:
df

,description,equipmentid,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,...,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time,tag


In [28]:
pd.DataFrame(columns=vals.columns)

,factor,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time


In [17]:
rooms

,buildingid,id,name
0,27,41,Evans_Dummy_Room


In [ ]:
df = pd.merge(points, rooms, left_on='roomid', right_on='id', suffixes=('_point', '_room'))

In [ ]:

df = pd.merge(df, vals, left_on='id_point', right_on='pointid', )
return df

In [ ]:
type(api.building_values_at_time('27', '00:00:00'))

In [74]:
merge_tables()

TypeError: merge_tables() missing 3 required positional arguments: 'points', 'rooms', and 'vals'

In [5]:
search_results = tools.get_room_comparison_results(searches)
search_results.head()

,name_room,pointid_valve,valve,pointid_temp1 (RM),temp1 (RM),pointid_temp2 (RMT),temp2 (RMT)
0,102,608,12.0,609,73.42,610,73.42
1,112,572,3.0,573,70.98,574,70.98
2,121,589,9.0,590,70.72,591,70.72
3,202,622,15.0,623,69.70,624,71.11
4,203,640,9.0,641,67.62,642,67.62


## Room Inspector

Graph view, after a room is clicked

In [4]:
searches = {
    'building': '4',
    'date': '2017-08-18',
    'timestamp': '00:00:00',
    'room': '102',
    'temp1': '73.42',
    'temp2': '73.42',
    'valve': '12.0',
}

search_results = tools.get_room_inspector_results(searches)
search_results.head()

,datetime,temp1 (RM),temp2 (RMT),valve
0,2017-08-18 00:00:00,73.42,73.42,12.0
1,2017-08-18 00:15:00,73.40,73.40,12.0
2,2017-08-18 00:30:00,73.40,73.40,12.0
3,2017-08-18 00:45:00,73.39,73.39,12.0
4,2017-08-18 01:00:00,73.37,73.37,12.0


### Plotting

In [5]:
from bokeh.plotting import (figure, show)
from bokeh.io import output_notebook
from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [6]:
def make_all_room_inspector_graphs(df):
    """Creates Bokeh plots for room inspector, once table cell clicked"""
    
    # tags is list of column names, excluding datetime
    tags = list(search_results.set_index('datetime').columns)
    
    plots = [make_room_inspector_graph(df, tag)
             for tag in tags]
    
    show(row(plots))

In [7]:
def make_room_inspector_graph(df, tag):
    """Creates Bokeh plots for room inspector, once table cell clicked"""
    p = figure(plot_width=300, plot_height=300, x_axis_type='datetime')
    p.line(df['datetime'], df[tag], line_width=2)
    p.xaxis.axis_label = "time"
    p.yaxis.axis_label = tag
    p.toolbar.logo = None
    return p

In [8]:
make_all_room_inspector_graphs(search_results)